In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
import razdel
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
import maru


In [67]:
def tokenize_with_razdel(text):
    return [token.text for token in razdel.tokenize(text)]

mystem = Mystem()
def lemmatize_with_mystem(text):
    return [lemma for lemma in mystem.lemmatize(text) if not lemma.isspace()]

pymorphy = MorphAnalyzer()
def lemmatize_with_pymorphy(tokens):
    return [pymorphy.parse(token)[0].normal_form for token in tokens]


idf_vectorizer = TfidfVectorizer()


maru_rnn = maru.get_analyzer('rnn')
def lemmatize_with_maru(tokens):
    return [morph.lemma for morph in maru_rnn.analyze(tokens)]

In [68]:
train = pd.read_csv('train.csv')
train.shape
test = pd.read_csv('test.csv')
test.shape

(112466, 2)

In [69]:
real = 0

In [70]:
if real == 0:
    X_train, X_test, y_train, y_test = train_test_split(train.text, train.label, test_size=0.20, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.33, random_state=42)
if real == 1:
    X_train = train.text
    X_test = test.text
    y_train = train.label

In [71]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((15070,), (15070,), (7424,), (7424,))

In [72]:
X_train = pd.DataFrame(X_train, columns=['text'])

In [73]:
# %%time
# texts = []
# for i in range(X_train.shape[0]):
#     #print (X_train.text[i:i+1].values)
#     text = str(X_train.text[i:i+1].values).lower()
#     #print ('low ---',text)
# #     regex = re.compile(r'(\W)\1+')
# #     text = regex.sub(r'\1', text)
#     #print ('re1 ---',text)
# #     regex = re.compile(r'[^\w\s]')
# #     text = regex.sub(r' ', text).strip()
#     #print ('re2 ---',text)
#     text = tokenize_with_razdel(text)
#     #print ('token -',text)
#     text = lemmatize_with_pymorphy(text)
#     #print ('lemma -',text)
#     text = " ".join(text)
#     texts.append(text)
#     #print (text)
#     #print ('')
# print ('done')

In [74]:
##%%time
#vectorizer = CountVectorizer(min_df = 3)
#vectorizer.fit_transform(texts).todense().shape

In [75]:
#X_train = texts

In [76]:
import tqdm
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import classification_report

In [77]:
def evaluate_vectorizer(vectorizer):
    X_train_vec = vectorizer.fit_transform(tqdm.tqdm_notebook(X_train.text.values, desc='Vectorizing train:'))
    X_test_vec = vectorizer.transform(tqdm.tqdm_notebook(X_test, desc='Vectorizing test:'))
    return X_train_vec, X_test_vec

In [20]:
#X_train_vec, X_test_vec = evaluate_vectorizer(CountVectorizer(min_df=2))

In [63]:
#answers = []

In [64]:
# model = LinearSVC(random_state=42)
# model.fit(X_train_vec, y_train)
# predictions = model.predict(X_test_vec)
# answers.append(predictions)
# print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.62      0.55      0.58      1440
           1       0.97      0.95      0.96      3278
           2       0.74      0.81      0.77      2706

   micro avg       0.82      0.82      0.82      7424
   macro avg       0.78      0.77      0.77      7424
weighted avg       0.82      0.82      0.82      7424



In [65]:
# model = OneVsOneClassifier(SVC(kernel='linear'))
# model.fit(X_train_vec, y_train)
# predictions = model.predict(X_test_vec)
# answers.append(predictions)
# print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.64      0.53      0.58      1440
           1       0.97      0.94      0.96      3278
           2       0.73      0.83      0.78      2706

   micro avg       0.82      0.82      0.82      7424
   macro avg       0.78      0.77      0.77      7424
weighted avg       0.82      0.82      0.82      7424



In [58]:
# answers = pd.DataFrame(answers).T

In [59]:
# from sklearn.neural_network  import MLPClassifier

In [61]:
# model = MLPClassifier()
# model.fit(answers,y_test)
# predictions = model.predict(answers)
# print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.62      0.55      0.58      1440
           1       0.97      0.95      0.96      3278
           2       0.74      0.81      0.77      2706

   micro avg       0.82      0.82      0.82      7424
   macro avg       0.78      0.77      0.77      7424
weighted avg       0.82      0.82      0.82      7424



In [ ]:
# model = LinearSVC(random_state=42)
#     #model = OneVsOneClassifier(SVC(kernel='linear'))
#     print ('begin training')
#  predictions = model.predict(X_test_vec)
   
#     model.fit(X, y_train)
#        #print(classification_report(y_test, predictions))

In [29]:
#predictions_COUNT = evaluate_vectorizer(CountVectorizer(min_df=2));


begin training


C:\Users\1INVESTOR\Anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [43]:
predictions = evaluate_vectorizer(TfidfVectorizer(min_df=2));


begin training


In [46]:
predictions_tokenizer = evaluate_vectorizer(TfidfVectorizer(min_df=2, tokenizer=tokenize_with_razdel));


begin training


In [78]:
tfidf_vectorizer = TfidfVectorizer(
    min_df=2, 
    tokenizer=lambda text: lemmatize_with_maru(tokenize_with_razdel(text)),
)
X_train_vec, X_test_vec = evaluate_vectorizer(tfidf_vectorizer)

In [83]:
X_train

,text
88984,2 или 3 - в данном случае спорно.
16009,П + П КОНФИДЕНЦЫАЛЬНО 093 280 93 53
8633,"Кажется, p(сказал что болен|на самом деле боле..."
88359,быстро работает... смех в том что БД в докере ...
60718,Мур-мур мой мышка!Цёмаю обнимаю тебя мой сладк...
48310,м З7 познакомлюсь с женщиной ... 0930983056
94846,"я читала одку книгу, и там высказывалась мысль..."
45501,HOROSHIY PAREN 27/173 ISHU DEVUSHKU DLYA S/O S...
17164,"логарифм от каждой записи, потом же по этим ло..."
33805,А о каком конкурсе вы говорите?


In [52]:
answers = pd.DataFrame(predictions_maru).reset_index()
answers.columns = ['id','label']
answers.to_csv('answers_svc_linear_maru.csv',index=False)

In [ ]:
sample = pd.read_csv('sample.csv')
sample.head(2)

In [ ]:
answers.head(2)